In [9]:
import cv2 as cv
import numpy
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [10]:
def load_and_process_image(path):
    img = cv.imread(path)
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_invert = cv.bitwise_not(img_gray)
    return img_invert

def find_and_crop_bounding_box(image):
    ret, thresh = cv.threshold(image, 127, 255, 0)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    #In case of multiple contours, take one with the biggest area
    x, y, width, height = cv.boundingRect(sorted(contours, key=cv.contourArea)[-1])
    image = image[y:y+height, x:x+width]
    return image

def vectorize(path):
    img = load_and_process_image(path)
    img = find_and_crop_bounding_box(img)
    img = cv.resize(img, (30, 30))
    img = img.reshape(900,)
    return img

In [11]:
label_map = {
    '+': 10,
    '-': 11,
    'times': 12,
    'forward_slash': 13,
    '(': 14,
    ')': 15
}

dataset = []

for symbol in os.listdir("dataset/"):
    try:
        label = int(symbol)
    except:
        label = label_map[symbol]
    
    print('Processing: {}'.format(symbol))
    for image in tqdm(os.listdir(os.path.join('dataset', symbol))):
        image_path = os.path.join('dataset', symbol, image)
        features = vectorize(image_path)
        dataset.append('{};{};{}'.format(features, image_path, label))

  1%|▏         | 44/3068 [00:00<00:06, 439.47it/s]

Processing: 8


  0%|          | 57/26520 [00:00<00:46, 568.44it/s]

Processing: 1


  0%|          | 64/26141 [00:00<00:40, 639.55it/s]

...

  2%|▏         | 63/3251 [00:00<00:05, 625.55it/s]

Processing: times


  0%|          | 45/14355 [00:00<00:32, 446.85it/s]

Processing: )


100%|██████████| 14355/14355 [00:24<00:00, 596.96it/s]


In [12]:
with open('dataset.csv', 'w') as out:
    out.write('\n'.join(dataset))